In [1]:
import sys, os
sys.path.append(os.pardir) # 親ディレクトリのファイルをインポートするための設定
from dataset.mnist import load_mnist

# 最初の呼び出しは数分待つ
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

#それぞれのデータの形状を出力
print(x_train.shape) #訓練画像6000枚
print(t_train.shape)
print(x_test.shape) #テスト画像1000枚
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [4]:
import sys, os
sys.path.append(os.pardir) # 親ディレクトリのファイルをインポートするための設定
import numpy as np
import pickle
from dataset.mnist import load_mnist

def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

# pickleファイルのsample_weight.pklに保存された学習済みの重みパラメータを読み込む
def init_network():
    with open("../ch03/sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    
    return network

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(a):
    c=np.max(a)
    exp_a=np.exp(a - c) # オーバーフロー対策
    sum_exp_a=np.sum(exp_a)
    y=exp_a/sum_exp_a
    
    return y

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    
    return y

x,t = get_data()
network = init_network()

accuracy_cnt=0
#1枚ずつ繰り返し計算する
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y) # 最も確率の高い要素のインデックスを取得
    if p == t[i]:
        accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))
    

Accuracy:0.9352


In [7]:
import sys,os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

# flatten=Trueであるため、1次元配列で格納
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)
img = x_train[0]
label = t_train[0]
print(label)

print(img.shape)
img_show(img)
img = img.reshape(28,28) # 形状を元の画像サイズに変形
print(img.shape)

img_show(img)

5
(784,)
(28, 28)


In [8]:
x, _ = get_data()
network = init_network()
W1,W2,W3 = network['W1'], network['W2'], network['W3']
print(x.shape)
print(x[0].shape)
print(W1.shape)
print(W2.shape)
print(W3.shape)


(10000, 784)
(784,)
(784, 50)
(50, 100)
(100, 10)


In [9]:
#batchサイズ100
x, t=get_data()
network=init_network()

batch_size=100
accuracy_cnt=0

#100単位の幅
for i in range(0, len(x), batch_size):
    x_batch=x[i:i+batch_size]
    y_batch=predict(network, x_batch)
    #argmax()で最大値のインデックスを返す 最も確率の高い数字を出力する
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352
